In [ ]:
!pip install spacy==3.5.0


In [4]:
!python -m spacy info

2023-07-09 18:39:51.794890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 18:39:53.270561: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.5.0                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.107+-x86_64-with-glibc2.31
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.5.0)        



In [5]:
import spacy
from spacy.tokens import DocBin # efficient, good
from tqdm import tqdm  # to show progress

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [16]:
import json
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [17]:
TRAIN_DATA

{'classes': ['MUSLIM-GIRL-NAME', 'MUSLIM-BOY-NAME'],
 'annotations': [['Once upon a time, in a beautiful garden called Janna, there lived a little girl named Kamila. Kamila was known for her complete kindness and virtue. She had a special bond with nature and would spend her days among the fragrant flowers, spreading happiness wherever she went.\r',
   {'entities': [[86, 93, 'MUSLIM-GIRL-NAME']]}],
  ['\r', {'entities': []}],
  ['One day, Kamila came across a group of girls playing under a majestic tree. Their names were Zahra, Fatema, and Areej. They were picking flowers and singing songs of cheerfulness. Kamila joined them, and together they created a beautiful melody that echoed through the garden.\r',
   {'entities': [[93, 98, 'MUSLIM-GIRL-NAME'],
     [100, 106, 'MUSLIM-GIRL-NAME'],
     [112, 118, 'MUSLIM-GIRL-NAME']]}],
  ['\r', {'entities': []}],
  ['As they continued their joyful activities, a young girl named Jasmin appeared. She had recently arrived from a distant land calle

In [18]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 35/35 [00:00<00:00, 1240.37it/s]


In [19]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2023-07-09 19:43:34.716802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [20]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

2023-07-09 19:43:43.153310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-07-09 19:43:46,499] [INFO] Set up nlp object from config
[2023-07-09 19:43:46,518] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-07-09 19:43:46,522] [INFO] Created vocabulary
[2023-07-09 19:43:46,524] [INFO] Finished initializing nlp object
[2023-07-09 19:43:46,674] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     27.50    0.00    0.00    0.00    0.00
 17     200        128.80    847.35  100.00  100.00  100.00 

In [21]:
nlp_ner = spacy.load("/content/model-best")

In [22]:
# input test data
doc = nlp_ner('''In the bustling city of Azaan, a young girl named Safiya wandered through the vibrant marketplaces.
As she strolled along, she crossed paths with a curious boy named Zayn, known for his adventurous spirit.
Together, Safiya and Zayn embarked on a thrilling quest to uncover the secrets of the ancient temple of Aisha.''')

In [23]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter